# Feature Extraction of Websites

In [1]:
#importing numpy and pandas which are required for data pre-processing
import numpy as np
import pandas as pd
from tqdm import tqdm
import re

<font size="4" > Loading Rawdata </font>

In [2]:
#loading the legitimate URLs data to dataframe
raw_data0 = pd.read_csv("raw_datasets2/dataset splitted/phishing_websites_2000_4000.txt", sep='delimiter', header=None)
raw_data0.columns = ['URLs']
raw_data0.head()

C:\Users\samee\AppData\Local\Temp\ipykernel_17848\3989710678.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  raw_data0 = pd.read_csv("raw_datasets2/dataset splitted/phishing_websites_2000_4000.txt", sep='delimiter', header=None)


,URLs
0,http://email.m-aut.ru/dhl/autodhl/dhl/delivery...
1,http://nabagejec1893.blogspot.com/
2,https://fj1gh.app.link/w7m5S2lYn5
3,http://osdwatracor.qsite.dk/122946/Apply%20For...
4,https://www.secureupdate.appleld.com.duilawyer...


<font size="5">We divide the protocol from the entire URL to seperate column. </font>

In [3]:
raw_data0['URLs'].str.split("://").head() #Here we divided the protocol from the entire URL. but need it to be divided it 
                                                 #seperate column

0    [http, email.m-aut.ru/dhl/autodhl/dhl/delivery...
1                  [http, nabagejec1893.blogspot.com/]
2                   [https, fj1gh.app.link/w7m5S2lYn5]
3    [http, osdwatracor.qsite.dk/122946/Apply%20For...
4    [https, www.secureupdate.appleld.com.duilawyer...
Name: URLs, dtype: object

In [4]:
seperation_of_protocol = raw_data0['URLs'].str.split("://",expand = True) #expand argument in the split method will give you a new column

In [5]:
seperation_of_protocol.head()

,0,1,2,3,4,5
0,http,email.m-aut.ru/dhl/autodhl/dhl/deliveryform.php,None,None,None,None
1,http,nabagejec1893.blogspot.com/,None,None,None,None
2,https,fj1gh.app.link/w7m5S2lYn5,None,None,None,None
3,http,osdwatracor.qsite.dk/122946/Apply%20For%20A%20...,None,None,None,None
4,https,www.secureupdate.appleld.com.duilawyeryork.com...,None,None,None,None


<font size="5">Domain name column can be further sub divided into domain_names as well as sub_domain_names. Similarly, address column can also be further sub divided into path, query_string, file................</font>

In [6]:
seperation_domain_name = seperation_of_protocol[1].str.split("/", n=1, expand=True)
 #split(seperator,no of splits according to seperator(delimiter),expand)

In [7]:
seperation_domain_name.columns=["domain_name","address"] #renaming columns of data frame

In [8]:
seperation_domain_name.shape

(1978, 2)

In [9]:
#Concatenation of data frames
splitted_data = pd.concat([seperation_of_protocol[0],seperation_domain_name],axis=1)

In [10]:
splitted_data.columns = ['protocol','domain_name','address']

In [11]:
splitted_data.head()

,protocol,domain_name,address
0,http,email.m-aut.ru,dhl/autodhl/dhl/deliveryform.php
1,http,nabagejec1893.blogspot.com,
2,https,fj1gh.app.link,w7m5S2lYn5
3,http,osdwatracor.qsite.dk,122946/Apply%20For%20A%20Payday%20Loan%20Onlin...
4,https,www.secureupdate.appleld.com.duilawyeryork.com,ap/65e7c3ac382597a?cmd=_update&dispatch=65e7c3...


# Feature Extraction

Feature-1: URL length

1.Some website have long URL to Hide the Suspicious Part. So,

If the length of the URL is greater than or equal 54 characters then the URL classified as phishing.


0 --> indicates legitimate

1 --> indicates Phishing

In [12]:
def long_url(l):
    """This function is defined in order to differntiate website based on the length of the URL"""
    if len(l) < 54:
        return 0
    else:
        return 1

In [13]:
#Applying the above defined function in order to divide the websites into 3 categories
splitted_data['long_url'] = raw_data0['URLs'].apply(long_url) 

In [14]:
#Will show the results only the websites which are legitimate according to above condition as 0 is legitimate website
splitted_data[splitted_data.long_url == 0].head()

,protocol,domain_name,address,long_url
1,http,nabagejec1893.blogspot.com,,0
2,https,fj1gh.app.link,w7m5S2lYn5,0
5,http,atelierba.com,data/log/?,0
9,http,actiie844.coffeecup.com,forms/info..w/,0
10,https,www.citieid.com,kenya,0


Feature-2

2.URL’s having “@” Symbol

Using “@” symbol in the URL leads the browser to ignore everything preceding the “@” symbol and the real address often follows the “@” symbol.

IF {Url Having @ Symbol→ Phishing
    Otherwise→ Legitimate }


0 --> indicates legitimate

1 --> indicates Phishing

In [15]:
def have_at_symbol(l):
    """This function is used to check whether the URL contains @ symbol or not"""
    if "@" in l:
        return 1
    return 0

In [16]:
splitted_data['having_@_symbol'] = raw_data0['URLs'].apply(have_at_symbol)

In [17]:
splitted_data.head()

,protocol,domain_name,address,long_url,having_@_symbol
0,http,email.m-aut.ru,dhl/autodhl/dhl/deliveryform.php,1,0
1,http,nabagejec1893.blogspot.com,,0,0
2,https,fj1gh.app.link,w7m5S2lYn5,0,0
3,http,osdwatracor.qsite.dk,122946/Apply%20For%20A%20Payday%20Loan%20Onlin...,1,0
4,https,www.secureupdate.appleld.com.duilawyeryork.com,ap/65e7c3ac382597a?cmd=_update&dispatch=65e7c3...,1,0


Feature-3

3.Redirecting using “//”

The existence of “//” within the URL path means that the user will be redirected to another website.
An example of such URL’s is: “http://www.example.com//http://www.realbank.com”. 
We examine the location where the “//” appears. 
We find that if the URL starts with “HTTP”, that means the “//” should appear in the sixth position. 
However, if the URL employs “HTTPS” then the “//” should appear in seventh position.


0 --> indicates Legitimate

1 --> indicates Phishing

In [18]:
def redirection(l):
    """If the url has symbol(//) after protocol then such URL is to be classified as phishing """
    if "//" in l:
        return 1
    return 0

In [19]:
splitted_data['redirection_//_symbol'] = seperation_of_protocol[1].apply(redirection)

In [20]:
splitted_data.head()

,protocol,domain_name,address,long_url,having_@_symbol,redirection_//_symbol
0,http,email.m-aut.ru,dhl/autodhl/dhl/deliveryform.php,1,0,0
1,http,nabagejec1893.blogspot.com,,0,0,0
2,https,fj1gh.app.link,w7m5S2lYn5,0,0,0
3,http,osdwatracor.qsite.dk,122946/Apply%20For%20A%20Payday%20Loan%20Onlin...,1,0,0
4,https,www.secureupdate.appleld.com.duilawyeryork.com,ap/65e7c3ac382597a?cmd=_update&dispatch=65e7c3...,1,0,0


Feature-4

4.Adding Prefix or Suffix Separated by (-) to the Domain

The dash symbol is rarely used in legitimate URLs. Phishers tend to add prefixes or suffixes separated by (-) to the domain name
so that users feel that they are dealing with a legitimate webpage. 

For example https://www.nabilbank.com is the real website but phishing websites could be https://www.real-nabils-banks.com which is phishing.
    
IF {Domain Name Part Includes (−) Symbol → Phishing
    
    Otherwise → Legitimate
    
1 --> indicates phishing

0 --> indicates legitimate
    

In [21]:
def prefix_suffix_seperation(l):
    if '-' in l:
        return 1
    return 0

In [22]:
splitted_data['prefix_suffix_seperation'] = seperation_domain_name['domain_name'].apply(prefix_suffix_seperation)

In [23]:
splitted_data.head()

,protocol,domain_name,address,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation
0,http,email.m-aut.ru,dhl/autodhl/dhl/deliveryform.php,1,0,0,1
1,http,nabagejec1893.blogspot.com,,0,0,0,0
2,https,fj1gh.app.link,w7m5S2lYn5,0,0,0,0
3,http,osdwatracor.qsite.dk,122946/Apply%20For%20A%20Payday%20Loan%20Onlin...,1,0,0,0
4,https,www.secureupdate.appleld.com.duilawyeryork.com,ap/65e7c3ac382597a?cmd=_update&dispatch=65e7c3...,1,0,0,0


Feature - 5

5. Sub-Domain and Multi Sub-Domains

The legitimate URL link mostly has two dots(.) in the URL since we can ignore typing “www.”. 
If the number of dots is equal to three then the URL is classified as “Legitimate” since it has one sub-domain.
However, if the dots are greater than three it is classified as “Phishing” since it will have multiple sub-domains

0 --> indicates legitimate

1 --> indicates Phishing

In [24]:
def sub_domains(l):
    if l.count('.') <= 3:
        return 0
    else:
        return 1

In [25]:
splitted_data['sub_domains'] = splitted_data['domain_name'].apply(sub_domains)

In [26]:
splitted_data.head()

,protocol,domain_name,address,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation,sub_domains
0,http,email.m-aut.ru,dhl/autodhl/dhl/deliveryform.php,1,0,0,1,0
1,http,nabagejec1893.blogspot.com,,0,0,0,0,0
2,https,fj1gh.app.link,w7m5S2lYn5,0,0,0,0,0
3,http,osdwatracor.qsite.dk,122946/Apply%20For%20A%20Payday%20Loan%20Onlin...,1,0,0,0,0
4,https,www.secureupdate.appleld.com.duilawyeryork.com,ap/65e7c3ac382597a?cmd=_update&dispatch=65e7c3...,1,0,0,0,1


Feature-6

6.Using the IP Address

If an IP address is used as an alternative of the domain name in the URL, such as “http://125.96.2.163/fake.html”,
users can be sure that someone is trying to steal their personal information. Sometimes, 
the IP address is even transformed into hexadecimal code as shown in the following link “http://0x58.0xCC.0xCA.0x62/2/payoneer.bd/index.html”.

Rule: IF{If The Domain Part has an IP Address → Phishing
        Otherwise→ Legitimate

 1 --> indicates phishing

 0 --> indicates legitimate

In [27]:
import re
def having_ip_address(url):
    match=re.search('(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  #IPv4
                    '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)'  #IPv4 in hexadecimal
                    '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}',url)     #Ipv6
    if match:
        #print match.group()
        return 1
    else:
        #print 'No matching pattern found'
        return 0

In [28]:
splitted_data['having_ip_address'] = raw_data0['URLs'].apply(having_ip_address)

In [29]:
splitted_data.head()

,protocol,domain_name,address,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation,sub_domains,having_ip_address
0,http,email.m-aut.ru,dhl/autodhl/dhl/deliveryform.php,1,0,0,1,0,0
1,http,nabagejec1893.blogspot.com,,0,0,0,0,0,0
2,https,fj1gh.app.link,w7m5S2lYn5,0,0,0,0,0,0
3,http,osdwatracor.qsite.dk,122946/Apply%20For%20A%20Payday%20Loan%20Onlin...,1,0,0,0,0,0
4,https,www.secureupdate.appleld.com.duilawyeryork.com,ap/65e7c3ac382597a?cmd=_update&dispatch=65e7c3...,1,0,0,0,1,0


Feature-7

7.Using URL Shortening Services “TinyURL”

URL shortening is a method on the “World Wide Web” in which a URL may be made considerably smaller in length and still lead to the required webpage. 
This is accomplished by means of an “HTTP Redirect” on a domain name that is short, which links to the webpage that has a long URL. 
For example, the URL “http://real-bank.com/login” can be shortened to “bit.ly/19DXSk4”.
    
Rule: IF{TinyURL → Phishing
         
         Otherwise→ Legitimate
         
                 1 --> indicates phishing

                 0 --> indicates legitimate

In [30]:
#we have imported re module in the above feature. So need not to import again
def shortening_service(url):
    match = re.search(r'bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      r'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      r'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      r'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      r'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      r'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      r'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|'
                      r'v\.gd|tr\.im|link\.zip\.net', url)
    if match:
        return 1
    else:
        return 0

In [31]:
splitted_data['shortening_service'] = raw_data0['URLs'].apply(shortening_service)

In [32]:
splitted_data.head()

,protocol,domain_name,address,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation,sub_domains,having_ip_address,shortening_service
0,http,email.m-aut.ru,dhl/autodhl/dhl/deliveryform.php,1,0,0,1,0,0,0
1,http,nabagejec1893.blogspot.com,,0,0,0,0,0,0,1
2,https,fj1gh.app.link,w7m5S2lYn5,0,0,0,0,0,0,0
3,http,osdwatracor.qsite.dk,122946/Apply%20For%20A%20Payday%20Loan%20Onlin...,1,0,0,0,0,0,0
4,https,www.secureupdate.appleld.com.duilawyeryork.com,ap/65e7c3ac382597a?cmd=_update&dispatch=65e7c3...,1,0,0,0,1,0,0


Feature - 8 

8.The Existence of “HTTPS” Token in the Domain Part of the URL

The phishers may add the “HTTPS” token to the domain part of a URL in order to trick users.
For example, http://https-www-paypal-it-webapps-mpp-home.soft-hair.com/.

Rule: IF{Using HTTP Token in Domain Part of The URL→ Phishing
             
Otherwise→ Legitimate

In [33]:
def https_token(url):
    match=re.search('https://|http://',url)
    if match.start(0)==0:
        url=url[match.end(0):]
    match=re.search('http|https',url)
    if match:
        return 1
    else:
        return 0

In [34]:
splitted_data['https_token'] = raw_data0['URLs'].apply(https_token)

In [35]:
splitted_data.head()
#!pip install retry

,protocol,domain_name,address,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation,sub_domains,having_ip_address,shortening_service,https_token
0,http,email.m-aut.ru,dhl/autodhl/dhl/deliveryform.php,1,0,0,1,0,0,0,0
1,http,nabagejec1893.blogspot.com,,0,0,0,0,0,0,1,0
2,https,fj1gh.app.link,w7m5S2lYn5,0,0,0,0,0,0,0,0
3,http,osdwatracor.qsite.dk,122946/Apply%20For%20A%20Payday%20Loan%20Onlin...,1,0,0,0,0,0,0,0
4,https,www.secureupdate.appleld.com.duilawyeryork.com,ap/65e7c3ac382597a?cmd=_update&dispatch=65e7c3...,1,0,0,0,1,0,0,0


Feature - 9

9.Website Traffic

This feature measures the popularity of the website by determining the number of visitors and the number of pages they visit.
However, since phishing websites live for a short period of time, they may not be recognized by the Alexa database 
(Alexa the Web Information Company., 1996). By reviewing our dataset, we find that in worst scenarios,
legitimate websites ranked among the top 100,000. Furthermore,
if the domain has no traffic or is not recognized by the Alexa database, it is classified as “Phishing”.

Rule: IF{Website Rank<100,000 → LegitimateWebsite Rank>100,000 → Phishing Website

The top 1Million sites for web traffic is downloaded from here https://tranco-list.eu/list/ZW96G/1000000

In [36]:
# Load Tranco top domains (ranked list)
top_domains = set()
with open("web_traffic_dataset/tranco_top_1m.csv") as f:
    for line in f:
        domain = line.strip().lower()
        top_domains.add(domain)

def web_traffic(domain):
    domain = domain.lower().replace("http://", "").replace("https://", "").split('/')[0]
    return 0 if domain in top_domains else 1  # 0 = high traffic, 1 = low traffic


In [37]:
tqdm.pandas()
try:
    splitted_data['web_traffic'] = raw_data0['URLs'].progress_apply(web_traffic)
    #x = web_traffic('www.google.com')
    #print(x)
except (ConnectionResetError,UnicodeEncodeError) as Error:
    print (Error)

100%|██████████████████████████████████████████████████████████████████████████████████████| 1978/1978 [00:00<?, ?it/s]


In [38]:
splitted_data.head()

,protocol,domain_name,address,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation,sub_domains,having_ip_address,shortening_service,https_token,web_traffic
0,http,email.m-aut.ru,dhl/autodhl/dhl/deliveryform.php,1,0,0,1,0,0,0,0,1
1,http,nabagejec1893.blogspot.com,,0,0,0,0,0,0,1,0,1
2,https,fj1gh.app.link,w7m5S2lYn5,0,0,0,0,0,0,0,0,1
3,http,osdwatracor.qsite.dk,122946/Apply%20For%20A%20Payday%20Loan%20Onlin...,1,0,0,0,0,0,0,0,1
4,https,www.secureupdate.appleld.com.duilawyeryork.com,ap/65e7c3ac382597a?cmd=_update&dispatch=65e7c3...,1,0,0,0,1,0,0,0,1


In [39]:
#!pip install python-whois

Feature - 10

10. Domain Registration Length

Based on the fact that a phishing website lives for a short period of time, 
we believe that trustworthy domains are regularly paid for several years in advance. 
In our dataset, we find that the longest fraudulent domains have been used for 6 months to one year only.

Rule: IF{Domains Expires on≤ 6 months → Phishing
        else Otherwise→ Legitimate}

In [40]:
# import whois
# from datetime import datetime
# import time
# from bs4 import BeautifulSoup
# from tqdm import tqdm
# from contextlib import suppress

# tqdm.pandas()

# def domain_registration_length_sub(domain):
#     expiration_date = domain.expiration_date
#     today = time.strftime('%Y-%m-%d')
#     today = datetime.strptime(today, '%Y-%m-%d')
#     if expiration_date is None:
#         return 1
#     elif type(expiration_date) is list or type(today) is list :
#         return 0             #If it is a type of list then we can't select a single value from list. So,it is regarded as suspected website  
#     else:
#         registration_length = abs((expiration_date - today).days)
#         if registration_length / 182 <= 1:
#             return 1
#         else:
#             return 0

In [41]:
# def domain_registration_length_main(domain):
#     dns = 0
#     try:
#         domain_name = whois.whois(domain)
#     except:
#         dns = 1
        
#     if dns == 1:
#         return 1
#     else:
#         return domain_registration_length_sub(domain_name)

In [42]:
# splitted_data['domain_registration_length'] = splitted_data['domain_name'].progress_apply(domain_registration_length_main)

In [ ]:
import whois
from datetime import datetime
import time
import socket
from retry import retry
import tldextract
from tqdm import tqdm

tqdm.pandas()

def domain_exists(domain):
    try:
        socket.gethostbyname(domain)
        return True
    except socket.gaierror:
        return False

@retry((Exception,), tries=3, delay=2, backoff=2)
def get_domain_info(domain):
    return whois.whois(domain)

def domain_registration_length(domain):
    # Extract registered domain (eTLD+1) from full domain
    ext = tldextract.extract(domain)
    registered_domain = ext.registered_domain  # e.g. blogspot.com

    if not registered_domain:
        # Could not extract domain, treat as suspicious
        return 1

    if not domain_exists(registered_domain):
        return 1

    try:
        time.sleep(1)  # Be gentle with WHOIS servers
        w = get_domain_info(registered_domain)
        exp_date = w.expiration_date

        if exp_date is None:
            return 1

        if isinstance(exp_date, list):
            exp_date = exp_date[0]

        if not isinstance(exp_date, datetime):
            return 1

        days_left = (exp_date - datetime.now()).days
        return 1 if days_left <= 182 else 0

    except Exception as e:
        print(f"[ERROR] Domain: {registered_domain}, Error: {e}")
        return 1

# Example applying to dataframe
splitted_data['domain_registration_length'] = splitted_data['domain_name'].progress_apply(domain_registration_length)


  1%|▌                                                                             | 14/1978 [00:33<2:11:44,  4.02s/it]

Error trying to connect to socket: closing socket - timed out


  1%|▉                                                                               | 24/1978 [00:49<58:08,  1.79s/it]

Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed


  5%|███▊                                                                          | 97/1978 [03:06<2:25:02,  4.63s/it]

Error trying to connect to socket: closing socket - timed out


  7%|█████                                                                        | 130/1978 [04:15<2:02:17,  3.97s/it]

Error trying to connect to socket: closing socket - timed out


  7%|█████▍                                                                       | 139/1978 [04:47<1:30:32,  2.95s/it]

In [ ]:
splitted_data
#splitted_data.to_csv('phishing_domain_registration_length.csv', index=False)

Feature - 11

11. DNS Record

For phishing websites, either the claimed identity is not recognized by the WHOIS database (Whois 2005) or 
no records founded for the hostname (Pan and Ding 2006). 
If the DNS record is empty or not found then the website is classified as “Phishing”, otherwise it is classified as “Legitimate”.

Rule: IF{no DNS Record For The Domain → Phishing
         
         Otherwise→ Legitimate}
        

In [ ]:
tqdm.pandas()
def dns_record(domain):
    dns = 0
    try:
        domain_name = whois.whois(domain)
        #print(domain_name)
    except:
        dns = 1
        
    if dns == 1:
        return 1
    else:
        return dns

In [ ]:
splitted_data['dns_record'] = splitted_data['domain_name'].progress_apply(dns_record)

In [ ]:
splitted_data

In [ ]:
##splitted_data.to_csv('legitimate_data2000_noageofdomain.csv', index=False)

Feature - 12: Age of Domain

This feature can be extracted from WHOIS database (Whois 2005). 
Most phishing websites live for a short period of time. 
By reviewing our dataset, we find that the minimum age of the legitimate domain is 6 months.

Rule: IF {Age Of Domain≥6 months → Legitimate
          
          Otherwise → Phishing

In [ ]:
from retry import retry
@retry((ValueError,TypeError), tries=3, delay=2)
def age_of_domain_sub(domain):
    try:
        creation_date = domain.creation_date
        expiration_date = domain.expiration_date
        print(creation_date)

        if(creation_date == "before Aug-1996"):
            creation_date = "1996-08-15"
            #print(datetime.strptime(creation_date,'%Y-%m-%d'))

        if ((expiration_date is None) or (creation_date is None)):
            return 1
        elif ((type(expiration_date) is list) or (type(creation_date) is list)):
            return 0
        else:
            ageofdomain = abs((expiration_date - creation_date).days)
            if ((ageofdomain/30) < 6):
                return 1
            else:
                return 0
    except Exception as TypeError:
        pass

In [ ]:
def age_of_domain_main(domain):
    dns = 0
    try:
        domain_name = whois.whois(domain)
        print(domain_name)
    except:
        dns = 1
        
    if dns == 1:
        return 1
    else:
        return age_of_domain_sub(domain_name)

In [ ]:
try:
    x = age_of_domain_main('metro.co.uk')
    print(x)
    #splitted_data['age_of_domain'] = splitted_data['domain_name'].progress_apply(age_of_domain_main)
except Exception as Error:
    print(Error)  

In [ ]:
splitted_data

In [ ]:
#splitted_data.to_csv('legitimate_data2000_all13_columns.csv', index=False)

Feature - 13

Statistical-Reports Based Feature

Several parties such as PhishTank (PhishTank Stats, 2010-2012),
and StopBadware (StopBadware, 2010-2012) formulate numerous statistical reports on phishing websites at every given period of time
some are monthly and others are quarterly. 

Rule: IF{Host Belongs to Top Phishing IPs or Top Phishing Domains → Phishing
         
         Otherwise → Legitimate


In [ ]:
import socket
def statistical_report(url):
    hostname = url
    h = [(x.start(0), x.end(0)) for x in re.finditer('https://|http://|www.|https://www.|http://www.', hostname)]
    z = int(len(h))
    if z != 0:
        y = h[0][1]
        hostname = hostname[y:]
        h = [(x.start(0), x.end(0)) for x in re.finditer('/', hostname)]
        z = int(len(h))
        if z != 0:
            hostname = hostname[:h[0][0]]
    url_match=re.search('at\.ua|usa\.cc|baltazarpresentes\.com\.br|pe\.hu|esy\.es|hol\.es|sweddy\.com|myjino\.ru|96\.lt|ow\.ly',url)
    try:
        ip_address = socket.gethostbyname(hostname)
        ip_match=re.search('146\.112\.61\.108|213\.174\.157\.151|121\.50\.168\.88|192\.185\.217\.116|78\.46\.211\.158|181\.174\.165\.13|46\.242\.145\.103|121\.50\.168\.40|83\.125\.22\.219|46\.242\.145\.98|107\.151\.148\.44|107\.151\.148\.107|64\.70\.19\.203|199\.184\.144\.27|107\.151\.148\.108|107\.151\.148\.109|119\.28\.52\.61|54\.83\.43\.69|52\.69\.166\.231|216\.58\.192\.225|118\.184\.25\.86|67\.208\.74\.71|23\.253\.126\.58|104\.239\.157\.210|175\.126\.123\.219|141\.8\.224\.221|10\.10\.10\.10|43\.229\.108\.32|103\.232\.215\.140|69\.172\.201\.153|216\.218\.185\.162|54\.225\.104\.146|103\.243\.24\.98|199\.59\.243\.120|31\.170\.160\.61|213\.19\.128\.77|62\.113\.226\.131|208\.100\.26\.234|195\.16\.127\.102|195\.16\.127\.157|34\.196\.13\.28|103\.224\.212\.222|172\.217\.4\.225|54\.72\.9\.51|192\.64\.147\.141|198\.200\.56\.183|23\.253\.164\.103|52\.48\.191\.26|52\.214\.197\.72|87\.98\.255\.18|209\.99\.17\.27|216\.38\.62\.18|104\.130\.124\.96|47\.89\.58\.141|78\.46\.211\.158|54\.86\.225\.156|54\.82\.156\.19|37\.157\.192\.102|204\.11\.56\.48|110\.34\.231\.42',ip_address)  
    except:
        return 1

    if url_match:
        return 1
    else:
        return 0

In [ ]:
try:
    splitted_data['statistical_report'] = raw_data0['URLs'].progress_apply(statistical_report)
except Exception as Error:
    print(Error)

In [ ]:
splitted_data

In [ ]:
splitted_data.to_csv('phishing_websites_2000_4000.csv', index=False)